In [ ]:
# %matplotlib inline
# from google.colab import files
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# from keras.models import Sequential
# from keras.layers.convolutional import Conv2D, MaxPooling2D

# from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix
import pickle
import onnx

print(tf.__version__)

In [ ]:
with open('X_test.pkl', 'rb') as file:
    X_test = pickle.load(file)

with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)

with open('X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)

with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(96, 96, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

In [ ]:
model.summary()

In [ ]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
history=model.fit(X_train, y_train, epochs=1, batch_size=64, verbose=1, validation_data=(X_test, y_test))

In [ ]:
model.save('handrecognition_model.h5')

In [ ]:
# model = tf.keras.models.load_model("handrecognition_model.h5")
# tf.saved_model.save(model, "tmp_model")

In [ ]:
# !python -m tf2onnx.convert --saved-model tmp_model --output "handrecognition_model.onnx"

In [ ]:

import tf2onnx
from onnx import checker

# Assume you have trained and saved your CNN model

# Load the trained model
model = keras.models.load_model('handrecognition_model.h5')

# Define the input spec
spec = (tf.TensorSpec((None, 96, 96, 1), tf.float32, name="input"),)

# Convert the model to ONNX format
output_path = "handrecognition_model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)

# Check the converted model
checker.check_graph(model_proto.graph)

In [ ]:
!zip -r tmp_model.zip tmp_model

In [ ]:
# from google.colab import files
# files.download("/content/handrecognition_model.h5")
# files.download("/content/handrecognition_model.onnx")
# files.download("/content/tmp_model.zip")